# Making chloropleth maps in Altair

Here's a quick example of how to make a chloropleth map in Altair.  In this example, we'll work with a fairly large data set of baby names in France from 1900-2019, broken down by department.

To work with geographical data, we'll use the `geopandas`, which loads `pandas` dataframes, but with support for geographical outlines in the `geojson` format.  You can use these dataframes just as you would a regular `pandas` dataframe, but they will include that extra geographical outline data.

To get started, we'll need to import our libraries.

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd # Requires geopandas -- e.g.: conda install -c conda-forge geopandas
alt.data_transformers.enable('json') # Let Altair/Vega-Lite work with large data sets

pass

# Reading our names data

Now, let's read in our dataset.  The exported data is in CSV format, but with a `;` separator instead of commas.  The INSEE data collapses rare names or where department-level information has been elided (presumably to protect individuals with uncommon names or who were one of the only ones born with that name in a given year).  We'll strip those out.

In [2]:
names = pd.read_csv("dpt2020.csv", sep=";")
names.drop(names[names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
names.drop(names[names.dpt == 'XX'].index, inplace=True)

names.sample(5)

,sexe,preusuel,annais,dpt,nombre
613805,1,GEORGES,1901,36,77
21615,1,ABOUBAKAR,2012,94,7
665791,1,GUILLAUME,1990,93,138
2248429,2,DANIELLE,1949,55,36
1038003,1,LUC,1941,38,3


# Loading map data

Next, let's load some map data of regions in France using `geopandas`.  These map data come from the [INSEE] and [IGN] and were processed into the `geojson` format we'll need to work with by [Grégoire David].  Here's the [github] repository.

In this example, we'll work with the simplified departments tiles for the Hexagon, but that repository contains higher-resolution versions, the DOM-TOM, and more.

[Grégoire David]: https://gregoiredavid.fr
[INSEE]: http://www.insee.fr/fr/methodes/nomenclatures/cog/telechargement.asp
[IGN]: https://geoservices.ign.fr/adminexpress
[github]: https://github.com/gregoiredavid/france-geojson/

In [3]:
depts = gpd.read_file('departements-version-simplifiee.geojson')

depts.sample(5)

,code,nom,geometry
76,76,Seine-Maritime,"POLYGON ((1.38155 50.06577, 1.40926 50.05707, ..."
90,90,Territoire de Belfort,"POLYGON ((6.82354 47.81305, 6.84618 47.82295, ..."
54,54,Meurthe-et-Moselle,"POLYGON ((5.47091 49.49721, 5.54118 49.51526, ..."
67,67,Bas-Rhin,"POLYGON ((7.63529 49.05416, 7.67449 49.04504, ..."
57,57,Moselle,"POLYGON ((5.8934 49.49691, 5.93994 49.50097, 5..."


Notice how `depts` is a geopandas dataframe.  We'll use it just as a regular `pandas` dataframe, but it includes the geometry info we need to be able to draw those regions when we pass them into Altair.  We just need to make sure that when we work with our data, we keep them in a geopandas dataframe and not a plain dataframe if we want to draw the departments.

In the next cell, notice how we do a right-merge to bring in department data into names.  We do this as a merge on `depts` because we need a geopandas dataframe.  Remember, `depts` is a geopandas dataframe, while `names` is a regular dataframe.  If we did a left merge on `names`, we'd end up with a regular pandas dataframe. After this merge, both `names` and `depts` will be geopandas dataframes.

**Hint:** Be careful when you do your data joins here.  It's easy to accidentally merge the wrong way to accidentally create a _much bigger_ dataset.

In [4]:
# Keep a reference around to the plain pandas dataframe, without geometry data, just in case
just_names = names

names = depts.merge(names, how='right', left_on='code', right_on='dpt')

names.sample(5)

,code,nom,geometry,sexe,preusuel,annais,dpt,nombre
170450,38,Isère,"POLYGON ((5.62375 45.61327, 5.62303 45.60428, ...",1,AUGUSTE,2010,38,6
2522866,29,Finistère,"MULTIPOLYGON (((-5.1026 48.43612, -5.1036 48.4...",2,HÉLÈNE,1917,29,37
1097146,88,Vosges,"POLYGON ((5.47006 48.42093, 5.51099 48.41822, ...",1,MARTIAL,1929,88,4
8010,NaN,NaN,None,1,ABEL,1918,971,11
343324,54,Meurthe-et-Moselle,"POLYGON ((5.47091 49.49721, 5.54118 49.51526, ...",1,DANIEL,2008,54,5


# Show a name over all years

Now we'll choose a name to show across all years.  To that, we'll group all of the names in a department together (squashing the years together) and use the sum.

In [5]:
grouped = names.groupby(['dpt', 'preusuel', 'sexe'], as_index=False).sum(numeric_only=True)
grouped = depts.merge(grouped, how='right', left_on='code', right_on='dpt') # Add geometry data back in
grouped

,code,nom,geometry,dpt,preusuel,sexe,nombre
0,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,AARON,1,160
1,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABBY,2,3
2,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDALLAH,1,7
3,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDEL,1,3
4,01,Ain,"POLYGON ((4.78021 46.17668, 4.79458 46.21832, ...",01,ABDELKADER,1,3
...,...,...,...,...,...,...,...
239574,NaN,NaN,None,974,ÉSAÏE,1,3
239575,NaN,NaN,None,974,ÉTHAN,1,53
239576,NaN,NaN,None,974,ÉTIENNE,1,3
239577,NaN,NaN,None,974,ÉVA,2,32


Now let's pick a name and check out how it's distribution over the last 120 years across Metropolitan France.  In this example, I choose the name “Lucien,” which I rather like for some reason.

In [6]:
multi = alt.selection_multi()

name = 'LUCIEN'
subset = grouped[grouped.preusuel == name]
alt.Chart(subset).mark_geoshape(stroke='white').encode(
    tooltip=['nom', 'code', 'nombre'],
    color='nombre',
).properties(width=800, height=600).add_selection(
    multi
)

C:\Users\mlarn\AppData\Local\Temp\ipykernel_42464\2301346975.py:1: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  multi = alt.selection_multi()
C:\Users\mlarn\AppData\Local\Temp\ipykernel_42464\2301346975.py:8: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).properties(width=800, height=600).add_selection(


alt.Chart(...)

In [7]:
import pandas as pd
import geopandas as gpd
import altair as alt

In [8]:
# Visualization 3
df_names = pd.read_csv('dpt2020.csv', sep=";", dtype={'dpt': str})
df_names.drop(df_names[df_names.preusuel == '_PRENOMS_RARES'].index, inplace=True)
df_names.drop(df_names[df_names.dpt == 'XX'].index, inplace=True)
gdf_depts = gpd.read_file('departements-version-simplifiee.geojson')

df_sex = df_names.groupby(['annais', 'sexe'])['nombre'].sum().reset_index()
df_sex['sexe'] = df_sex['sexe'].map({1: 'Male', 2: 'Female'})

alt.Chart(df_sex).mark_line().encode(
    x='annais:O',
    y='nombre:Q',
    color='sexe:N'
).properties(
    title='Total Name Counts by Sex Over Time'
)


alt.Chart(...)

In [11]:
import numpy as np
s1 = df_names[df_names['sexe'] == 1]['preusuel'].unique()
s2 = df_names[df_names['sexe'] == 2]['preusuel'].unique()
unisex_names = np.intersect1d(s1, s2)
df_unisex = df_names[df_names['preusuel'].isin(unisex_names)]
df_unisex_1 = df_unisex[df_unisex['sexe'] == 1]
df_unisex_2 = df_unisex[df_unisex['sexe'] == 2]

top_names_1 = df_unisex_1.groupby('preusuel')['nombre'].sum().nlargest(200).index
top_names_2 = df_unisex_2.groupby('preusuel')['nombre'].sum().nlargest(200).index
top_names = np.intersect1d(top_names_1, top_names_2)
df_unisex_filtered = df_unisex[df_unisex['preusuel'].isin(top_names)].groupby(['preusuel', 'annais']).sum().reset_index()

alt.Chart(df_unisex_filtered).mark_line().encode(
    x=alt.X('annais:T', title='Year'),
    y=alt.Y('nombre:Q', title='Number'),
    color='preusuel:N',
    tooltip=[
        alt.Tooltip('preusuel:N', title='Name'), 
        alt.Tooltip('year(annais):O', title='Year'), 
        alt.Tooltip('nombre:Q', title='Number')]
).properties(
    title='Top unisex Names Over Time'
).properties(
    width=600, height=300
)

alt.Chart(...)

In [12]:
name = 'CAMILLE'
df_name = df_names[df_names['preusuel'] == name]
df_name = df_name.groupby(['sexe', 'annais']).sum().reset_index()

alt.Chart(df_name).mark_line().encode(
    x=alt.X('year(annais):T', title='Year'),
    y=alt.Y('nombre:Q', title='Number'),
    color='sexe:N',
    tooltip=[
        alt.Tooltip('preusuel:N', title='Name'), 
        alt.Tooltip('year(annais):O', title='Year'), 
        alt.Tooltip('nombre:Q', title='Number')]
).properties(
    title=f'Evolution of the name {name} over time for both sexes'
).properties(
    width=600, height=300
)

alt.Chart(...)